In [1]:
import json
import logging
import os
import sys
from pathlib import Path
from collections import defaultdict
from pathlib import Path
import numpy as np
import re
from os import path
import warnings
warnings.filterwarnings("ignore")
from cli.cli_parser import parse_args
from graph.graph import AssociatedGraph
from utils.preprocessing import create_graphs
from itertools import combinations
import pandas as pd

In [2]:
class obj:
    def __init__(self, dict1):
        self.__dict__.update(dict1)
 
def dict2obj(dict1):
    return json.loads(json.dumps(dict1), object_hook=obj)

In [3]:
args = dict2obj({
        "folder_path": "Analysis/selected_strs_renumber/without_TCR",
        "residues_lists": "jsons/residues_lists.json",
        "serd_config": None,
        "files_name": None,
        "output_path": None,
        "run_name": None,
        "check_depth": False,
        "check_rsa": True,
        "centroid_threshold": 8.5,
        "distance_diff_threshold": 3,
        "depth_filter": None,
        "depth_bins": 3,
        "rsa_filter": 0.1,
        "rsa_bins": 3,
        "distance_bins": 3,
        "factors_path": "resources/atchley_aa.csv",
        "classes_path": None})

checks = {
        "depth": args.check_depth,
        "rsa": args.check_rsa,
}

association_config = {
    "centroid_threshold": args.centroid_threshold,
    "distance_diff_threshold": args.distance_diff_threshold,
    "rsa_filter": args.rsa_filter,
    "depth_filter": args.depth_filter,
    "rsa_bins": args.rsa_bins,
    "depth_bins": args.depth_bins,
    "distance_bins": args.distance_bins,
    "checks": checks,
    "factors_path": args.factors_path,
    "classes_path": args.classes_path
}



In [4]:
diffMHC_diffPep = pd.read_csv("Analysis/crossreact_processed_diff_MHC_diff_pep_helder.csv")
diffMHC_SamePep = pd.read_csv("Analysis/crossreact_processed_diff_MHC_same_pep_helder.csv")
sameMHC_diffPep = pd.read_csv("Analysis/crossreact_processed_same_MHC_diff_pep_helder.csv")
rawCross = pd.read_csv("Analysis/crossreact_tcrs_v3_blast.csv")

In [5]:
rawCross

,TCR_ID,TRA,TRB,peptide,MHCseq,MHCseq_ref,allele,allele_blast,mismatches,Score,...,TRA_ref,TRB_ref,TCR_pair_id,MHC_allele_id,MHC_allele_id_interface,peptide_id,pMHC_id,pep_crossreact,mhc_crossreact,pmhc_crossreact
0,PDB4ms8,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,SPAEAGFFL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,H-2Ld,"F9Y,V13T,P16R,I24T,N31D,A50V,K132R",3,...,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,0,0,0,3,3,True,False,False
1,PDB4mxq,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,SPAPRPLDL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,H-2Ld,"F9Y,V13T,P16R,I24T,N31D,A50V,K132R",3,...,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,0,0,0,17,18,True,False,False
2,PDB4n0c,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,MPAGRPWDL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,H-2Ld,"F9Y,V13T,P16R,I24T,N31D,A50V,K132R",3,...,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,0,0,0,54,58,True,False,False
3,PDB4n5e,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,VPYMAEFGM,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,H-2Ld,"F9Y,V13T,P16R,I24T,N31D,A50V,K132R",3,...,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,0,0,0,0,0,True,False,False
4,PDB4mvb,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,QPAEGGFQL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,H-2Ld,"F9Y,V13T,P16R,I24T,N31D,A50V,K132R",3,...,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,0,0,0,19,20,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,PDB7dzm,DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQY...,AGVIQSPRHEVTEMGQEVTLRCKPISGHNSLFWYRQTMMRGLELLI...,TPQDLNTML,GGSHSMRYFYTSVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPRE...,GGSHSMRYFYTSVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPRE...,NaN,B*81:02,A1G,3,...,DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQY...,AGVIQSPRHEVTEMGQEVTLRCKPISGHNSLFWYRQTMMRGLELLI...,34,23,23,71,78,False,True,False
107,PDB7dzn,DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQY...,DAGVIQSPRHEVTEMGQEVTLRCKPISGHNSLFWYRQTMMRGLELL...,TPQDLNTML,MGGSHSMRYFYTSVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPR...,GGSHSMRYFYTSVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPRE...,NaN,B*42:30,A2G,3,...,DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQY...,AGVIQSPRHEVTEMGQEVTLRCKPISGHNSLFWYRQTMMRGLELLI...,34,25,24,71,85,False,True,False
108,PDB8eo8,GENVEQHPSTLSVQEGDSAVIKCTYSDSASNYFPWYKQELGKRPQL...,AVVSQHPSRVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLMLM...,LPFDKATIM,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,B*35:01:01:01,NaN,NaN,3,...,GENVEQHPSTLSVQEGDSAVIKCTYSDSASNYFPWYKQELGKRPQL...,VVSQHPSRVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLMLMA...,35,11,12,84,94,True,False,False
109,PDB8enh,GENVEQHPSTLSVQEGDSAVIKCTYSDSASNYFPWYKQELGKRPQL...,AVVSQHPSRVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLMLM...,LPFEKSTIM,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,B*35:01:01:01,NaN,NaN,3,...,GENVEQHPSTLSVQEGDSAVIKCTYSDSASNYFPWYKQELGKRPQL...,VVSQHPSRVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLMLMA...,35,11,12,64,71,True,False,False


In [6]:
crossDf = rawCross[["TCR_ID", "TCR_pair_id", "MHC_allele_id", "peptide_id", "pMHC_id", "PDB_ID"]]
crossDf

,TCR_ID,TCR_pair_id,MHC_allele_id,peptide_id,pMHC_id,PDB_ID
0,PDB4ms8,0,0,3,3,4MS8
1,PDB4mxq,0,0,17,18,4MXQ
2,PDB4n0c,0,0,54,58,4N0C
3,PDB4n5e,0,0,0,0,4N5E
4,PDB4mvb,0,0,19,20,4MVB
...,...,...,...,...,...,...
106,PDB7dzm,34,23,71,78,7DZM
107,PDB7dzn,34,25,71,85,7DZN
108,PDB8eo8,35,11,84,94,8EO8
109,PDB8enh,35,11,64,71,8ENH


In [7]:
if (n := len([1, 2, 3])) > 2:
    print(f"A lista tem {n} elementos")


A lista tem 3 elementos


In [8]:
TCR_pair_id = crossDf.groupby('TCR_pair_id')
Graphs = {}
for pair_id, group in TCR_pair_id:
    print(f"\n--- TCR_pair_id: {pair_id} ---")
    args.files_name = ",".join([f"noTCR_{name.lower()}.trunc.fit_renum.pdb" for name in group["PDB_ID"]])
    args.run_name = str(pair_id)
    args.output_path = f"Analysis/CrossGraphs/{pair_id}/All"

    graphs = create_graphs(args)
    try:
        G = AssociatedGraph(
            graphs=graphs,
            output_path=args.output_path,
            run_name=args.run_name,
            association_config=association_config
        )
    except Exception as e:
        print(f"Não foi possível criar grafo: {e}")
    else:
        G.draw_graph_interactive(show=False, save=True)
        Graphs[pair_id] = G
        graphs_raw = G.graph_data

        
        associated_graph_data = dict()
        associated_graph_data["original_graphs"] = {}

        for graph_raw in graphs_raw:
            pdb_file = graph_raw["pdb_file"]
            _id = graph_raw["id"]

            if not (m := re.search(r'noTCR_([A-Za-z0-9]{4})\.trunc', pdb_file, re.IGNORECASE)):
                print(f"[ERRO] Código PDB não encontrado em: {pdb_file}")
                continue
            name = m[1]
            
            nodes = list(graph_raw["graph"].nodes)
            edges = list(graph_raw["graph"].edges)

            neighbors = {
                str(node): [str(neighbor) for neighbor in graph_raw["graph"].neighbors(node)]
                for node in nodes     
            }

            associated_graph_data["original_graphs"][_id] = {
                "name": name,
                "nodes": nodes,
                "edges": edges,
                "neighbors": neighbors
            }

        for j, comps in enumerate(G.associated_graphs):
            associated_graph_data[j] = {"comp": j, "frames": {}}
            for i in range(len(comps[0])):
                nodes = list(comps[0][i].nodes)
                edges = list(comps[0][i].edges)
                neighbors = {
                    str(node): [str(neighbor) for neighbor in comps[0][i].neighbors(node)]
                    for node in nodes     
                    }

                associated_graph_data[j]["frames"][i] = {
                    "nodes": nodes,
                    "edges": edges,
                    "neighbors": neighbors
                }

        output_json = path.join(args.output_path, f"graph_{args.run_name}.json")
        with open(output_json, "w") as f:
            json.dump(associated_graph_data, f, indent=4)
        print(f"Graph data saved to {output_json}")




--- TCR_pair_id: 0 ---


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!


...

Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
Graph data saved to Analysis/CrossGraphs/34/All/graph_34.json

--- TCR_pair_id: 35 ---


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
Graph data saved to Analysis/CrossGraphs/35/All/graph_35.json


In [9]:
Graphs

{0: <graph.graph.AssociatedGraph at 0x7fb0d4b9aa90>,
 1: <graph.graph.AssociatedGraph at 0x7fb0cc660a30>,
 2: <graph.graph.AssociatedGraph at 0x7fb0cc660e20>,
 3: <graph.graph.AssociatedGraph at 0x7fb0d3764a30>,
 4: <graph.graph.AssociatedGraph at 0x7fb0c6e4e730>,
 5: <graph.graph.AssociatedGraph at 0x7fb0cc9f4100>,
 6: <graph.graph.AssociatedGraph at 0x7fb0cc75fdf0>,
 7: <graph.graph.AssociatedGraph at 0x7fb0c74c9310>,
 8: <graph.graph.AssociatedGraph at 0x7fb0ada66be0>,
 9: <graph.graph.AssociatedGraph at 0x7fb0ada669d0>,
 10: <graph.graph.AssociatedGraph at 0x7fb0d41a23a0>,
 11: <graph.graph.AssociatedGraph at 0x7fb0d397f790>,
 12: <graph.graph.AssociatedGraph at 0x7fb09e0d36d0>,
 13: <graph.graph.AssociatedGraph at 0x7fb0cc457cd0>,
 14: <graph.graph.AssociatedGraph at 0x7fb0a82d7b80>,
 15: <graph.graph.AssociatedGraph at 0x7fb09df49b80>,
 16: <graph.graph.AssociatedGraph at 0x7fb0b37d4df0>,
 17: <graph.graph.AssociatedGraph at 0x7fb0d4416460>,
 18: <graph.graph.AssociatedGraph at 0

In [10]:
def load_data(json_path):
    return json.loads(Path(json_path).read_text())

def get_protein_keys(original_graphs: dict):
    keys = list(original_graphs.keys())
    if all(isinstance(k, str) and k.isdigit() for k in keys):
        return [str(i) for i in sorted(map(int, keys))]
    return keys

# ---------- projeção de nós ----------
def project_nodes_unique(frame_nodes, p):
    """Conjunto de nós únicos do frame projetado na proteína p."""
    return set(n[p] for n in frame_nodes)

def project_nodes_instances(frame_nodes, p):
    """Lista (com repetição) de nós do frame projetados na proteína p."""
    return [n[p] for n in frame_nodes]

# ---------- métrica por proteína (somente nós) ----------
def node_similarity_for_protein(frame, original_graphs, protein_keys, p):
    """
    frame: {"nodes":[...], "edges":[...]} (edges ignoradas aqui)
    p: índice posicional dentro de cada nó associado
    """
    nodes_assoc = frame.get("nodes", [])
    if not nodes_assoc:
        return None

    prot_key = protein_keys[p]
    og = original_graphs[prot_key]
    prot_name = og.get("name", prot_key)

    Vp = set(og["nodes"]) 

    inst = project_nodes_instances(nodes_assoc, p)
    Up   = set(inst)

    total_orig = len(Vp) if Vp else 0
    node_coverage = (len(Up) / total_orig) if total_orig else 0.0

    # indicador de repetição dentro do frame
    total_inst = len(inst)
    unique_cnt = len(Up)
    
    duplication_ratio = (len(Up) / total_inst) if total_inst else 1.0
    duplication_rate  = 1.0 - duplication_ratio
    avg_multiplicity  = (total_inst / unique_cnt) if unique_cnt else float('inf')

    return dict(
        protein_index=p,
        protein_key=prot_key,
        protein_name=prot_name,
        total_nodes_associated=len(nodes_assoc),
        total_nodes_original=total_orig,
        frame_nodes_instances=total_inst,
        frame_nodes_unique=len(Up),
        node_coverage=node_coverage,
        duplication_ratio=duplication_ratio,
        duplication_rate=duplication_rate,     # ∈[0,1),  0=sem duplicação
        avg_multiplicity=avg_multiplicity      # ≥1,       1=sem duplicação
    )

def wmean(x, w):
    x = np.asarray(x, float); w = np.asarray(w, float)
    s = w.sum()
    return float(np.sum(x*w)/s) if s > 0 else np.nan

def wstd(x, w):
    x = np.asarray(x, float); w = np.asarray(w, float)
    m = wmean(x, w)
    s = w.sum()
    return float(np.sqrt(np.sum(w*(x-m)**2)/s)) if s > 0 else np.nan

def wmedian(x, w):
    x = np.asarray(x, float); w = np.asarray(w, float)
    if w.sum() == 0: return np.nan
    order = np.argsort(x); x = x[order]; w = w[order]
    cw = np.cumsum(w)/w.sum()
    return float(x[np.searchsorted(cw, 0.5)])

def wtrimmed_mean(x, w, trim=0.10):
    x = np.asarray(x, float); w = np.asarray(w, float)
    if w.sum() == 0: return np.nan
    order = np.argsort(x); x = x[order]; w = w[order]
    cw = np.cumsum(w)/w.sum()
    keep = (cw >= trim) & (cw <= 1.0-trim)
    if not np.any(keep): keep = np.ones_like(cw, dtype=bool)
    return wmean(x[keep], w[keep])

def ivw_mean_proportions(cov, n):
    """
    Meta-análise de proporções (Jeffreys smoothing).
    cov: array de coberturas c_p, n: tamanhos |V_p|.
    """
    cov = np.asarray(cov, float); n = np.asarray(n, float)
    # sucessos aproximados = cov*n; Jeffreys: +0.5 / +1
    p = ((cov*n) + 0.5) / (n + 1.0)
    var = p*(1.0-p) / (n + 1.0) + 1e-12
    w = 1.0/var
    return wmean(p, w)

# ---------- RESUMO por frame (só nós) ----------
def summarize_frame_nodes(df_fp_nodes_for_frame):
    """
    Espera um DataFrame com colunas:
      - node_coverage  (|U_p|/|V_p|)
      - total_nodes_original (= |V_p|)
      - duplication_ratio   (opcional, só para reporte)
    """
    if df_fp_nodes_for_frame.empty:
        return {}
    cov = df_fp_nodes_for_frame["node_coverage"].values
    n   = df_fp_nodes_for_frame["total_nodes_original"].values
    w   = n

    return {
        # médias ponderadas
        "node_cov_wmean":       wmean(cov, w),
        "node_cov_wmedian":     wmedian(cov, w),
        "node_cov_wtrimmed":    wtrimmed_mean(cov, w, trim=0.10),
        "node_cov_ivw_meta":    ivw_mean_proportions(cov, n),

        # robustez / consistência
        "node_cov_wstd":        wstd(cov, w),
        "node_cov_p10":         float(np.percentile(cov, 10)),
        "node_cov_p50":         float(np.percentile(cov, 50)),
        "node_cov_p90":         float(np.percentile(cov, 90)),

        # diagnóstico
        "n_proteins":           int(len(cov)),
        "mean_dup_rate":       float(df_fp_nodes_for_frame.get("duplication_rate", pd.Series([np.nan])).mean()),
        "mean_graph_size":      float(np.mean(n)),
        "sum_graph_size":       int(np.sum(n)),
    }

# ---------- avaliação de UM frame ----------
def evaluate_frame_nodes(component_id, frame_id, data):
    comp_key = str(component_id)
    frm_key  = str(frame_id)
    frame = data[comp_key]["frames"][frm_key]

    original_graphs = data["original_graphs"]
    protein_keys = get_protein_keys(original_graphs)

    n_nodes_frame = len(frame["nodes"])
    n_prot = len(frame["nodes"][0]) if frame.get("nodes") else 0
    rows = []
    for p in range(n_prot):
        r = node_similarity_for_protein(frame, original_graphs, protein_keys, p)
        if r is not None:
            rows.append(r)

    df = pd.DataFrame(rows)
    if df.empty:
        summary = {}
    else:
        summary = dict(
            component=component_id,
            frame=frame_id,
            n_proteins=len(df),
            node_cov_mean=float(df["node_coverage"].mean()),
            node_cov_median=float(df["node_coverage"].median()),
            node_cov_min=float(df["node_coverage"].min()),
            node_cov_std=float(df["node_coverage"].std(ddof=0) if len(df)>1 else 0.0),
            dup_ratio_mean=float(df["duplication_ratio"].mean()),
        )
    return df, summary

# ---------- avaliação de TODOS os frames (somente nós) ----------
def evaluate_all_frames_nodes(json_path):
    data = load_data(json_path)
    component_ids = [k for k in data.keys() if k != "original_graphs"]
    try:
        component_ids = sorted(component_ids, key=lambda x: int(x))
    except Exception:
        pass

    all_fp, summaries = [], []
    for comp_id in component_ids:
        frames = data[comp_id]["frames"]
        frame_ids = list(frames.keys())
        try:
            frame_ids = sorted(frame_ids, key=lambda x: int(x))
        except Exception:
            pass

        for frm_id in frame_ids:
            df_fp, summ = evaluate_frame_nodes(comp_id, frm_id, data)
            if not df_fp.empty:
                df_fp.insert(0, "component_id", comp_id)
                df_fp.insert(1, "frame_id", frm_id)
                all_fp.append(df_fp)
            if summ:
                summaries.append(summ)

    df_fp_nodes = pd.concat(all_fp, ignore_index=True) if all_fp else pd.DataFrame()
    df_frames_nodes = pd.DataFrame(summaries).sort_values(
        ["node_cov_mean","node_cov_median","node_cov_min","node_cov_std"],
        ascending=[False, False, False, True]
    )
    return df_fp_nodes, df_frames_nodes

def evaluate_all_frames_nodes_weighted(json_path):
    # usa evaluate_all_frames_nodes(...) que você já tem para gerar df_fp_nodes e df_frames_nodes "simples"
    df_fp_nodes, _ = evaluate_all_frames_nodes(json_path)

    summaries = []
    if df_fp_nodes.empty:
        return df_fp_nodes, pd.DataFrame()

    for (comp_id, frame_id), g in df_fp_nodes.groupby(["component_id", "frame_id"], dropna=False):
        s = summarize_frame_nodes(g)
        s.update({"component_id": comp_id, "frame_id": frame_id})
        summaries.append(s)

    df_frames_nodes_w = pd.DataFrame(summaries)

    cols = ["component_id", "frame_id"] + [c for c in df_frames_nodes_w.columns if c not in ("component_id", "frame_id")]
    df_frames_nodes_w = df_frames_nodes_w[cols]

    # ranking: alta cobertura ponderada, alta mediana/percentil baixo alto, baixa dispersão
    df_frames_nodes_w = df_frames_nodes_w.sort_values(
        ["node_cov_wmean","node_cov_wmedian","node_cov_p10","node_cov_wstd"],
        ascending=[False, False, False, True]
    )
    return df_fp_nodes, df_frames_nodes_w


# ---------- ranking (nós) ----------
def rank_frames_by_node_coverage(df_frames_nodes, top=20):
    return df_frames_nodes.sort_values(
        ["node_cov_mean","node_cov_median","node_cov_min","node_cov_std"],
        ascending=[False, False, False, True]
    ).head(top)


In [11]:
df_fp_nodes, df_frames_nodes_w = evaluate_all_frames_nodes_weighted("Analysis/CrossGraphs/0/All/graph_0.json")
# # tabela por proteína de um frame
df_fp_nodes[(df_fp_nodes.component_id=='1') & (df_fp_nodes.frame_id=='0')][
    ["component_id", "frame_id", "protein_key","protein_name","node_coverage","frame_nodes_unique", "total_nodes_associated", "total_nodes_original","duplication_ratio"]
].sort_values("node_coverage", ascending=False)
# # ranking geral


,component_id,frame_id,protein_key,protein_name,node_coverage,frame_nodes_unique,total_nodes_associated,total_nodes_original,duplication_ratio
5,1,0,0,4ms8,0.415254,49,49,118,1.0
7,1,0,2,4n0c,0.392000,49,49,125,1.0
8,1,0,3,4n5e,0.392000,49,49,125,1.0
6,1,0,1,4mxq,0.385827,49,49,127,1.0
9,1,0,4,4mvb,0.368421,49,49,133,1.0


In [16]:
df_fp_nodes

,component_id,frame_id,protein_index,protein_key,protein_name,total_nodes_original,frame_nodes_instances,frame_nodes_unique,node_coverage,duplication_ratio,duplication_rate,avg_multiplicity
0,0,0,0,0,4ms8,118,118,46,0.389831,0.389831,0.610169,2.565217
1,0,0,1,1,3tjh,125,118,47,0.376000,0.398305,0.601695,2.510638
2,0,0,2,2,4mxq,127,118,47,0.370079,0.398305,0.601695,2.510638
3,0,0,3,3,3tfk,127,118,47,0.370079,0.398305,0.601695,2.510638
4,0,0,4,4,4n0c,125,118,47,0.376000,0.398305,0.601695,2.510638
5,0,0,5,5,4n5e,125,118,47,0.376000,0.398305,0.601695,2.510638
6,0,0,6,6,4mvb,133,118,46,0.345865,0.389831,0.610169,2.565217
7,0,0,7,7,3tpu,129,118,48,0.372093,0.406780,0.593220,2.458333
8,1,0,0,0,4ms8,118,42,42,0.355932,1.000000,0.000000,1.000000
9,1,0,1,1,3tjh,125,42,42,0.336000,1.000000,0.000000,1.000000


In [14]:
df_frames_nodes_w.head(20)

,component_id,frame_id,node_cov_wmean,node_cov_wmedian,node_cov_wtrimmed,node_cov_ivw_meta,node_cov_wstd,node_cov_p10,node_cov_p50,node_cov_p90,n_proteins,mean_dup_rate,mean_graph_size,sum_graph_size
0,0,0,0.371655,0.372093,0.369248,0.372511,0.011575,0.362815,0.374047,0.380149,8,0.602754,126.125,1009
1,1,0,0.333003,0.330709,0.329966,0.334155,0.010595,0.322644,0.333354,0.341980,8,0.000000,126.125,1009
2,1,1,0.190287,0.188976,0.188552,0.192580,0.006054,0.184368,0.190488,0.195417,8,0.000000,126.125,1009
4,1,3,0.182359,0.181102,0.180696,0.184717,0.005802,0.176686,0.182551,0.187275,8,0.000000,126.125,1009
3,1,2,0.174430,0.173228,0.172840,0.176855,0.005550,0.169004,0.174614,0.179132,8,0.000000,126.125,1009


In [62]:
rank_frames_by_node_coverage(df_frames_nodes, top=20)


,component,frame,n_proteins,node_cov_mean,node_cov_median,node_cov_min,node_cov_std,dup_ratio_mean
0,0,0,8,0.371993,0.374047,0.345865,0.011491,0.397246
1,1,0,8,0.333340,0.333354,0.315789,0.010687,1.000000
2,1,1,8,0.190480,0.190488,0.180451,0.006107,1.000000
4,1,3,8,0.182543,0.182551,0.172932,0.005852,1.000000
3,1,2,8,0.174607,0.174614,0.165414,0.005598,1.000000


In [12]:
def process_one_json(json_path: Path):
    """Roda a avaliação para um JSON e retorna (df_fp_nodes, df_frames_nodes_w)."""
    df_fp_nodes, df_frames_nodes_w = evaluate_all_frames_nodes_weighted(str(json_path))
    # compat: renomear/alinhar nomes que você já usa em filtros
    if "frame_nodes_unique" in df_fp_nodes.columns and "total_nodes_associated" not in df_fp_nodes.columns:
        df_fp_nodes["total_nodes_associated"] = df_fp_nodes["frame_nodes_unique"]
    return df_fp_nodes, df_frames_nodes_w

def save_pair_outputs(pair_dir: Path, df_fp_nodes: pd.DataFrame, df_frames_nodes_w: pd.DataFrame):
    """Salva os CSVs no diretório da análise (pair)."""
    out_dir = pair_dir / "All"
    out_dir.mkdir(parents=True, exist_ok=True)
    df_fp_nodes.to_csv(out_dir / "nodes_per_protein.csv", index=False)
    df_frames_nodes_w.to_csv(out_dir / "nodes_summary_weighted.csv", index=False)

def batch_evaluate_all(root="Analysis/CrossGraphs"):
    """
    Procura por Analysis/CrossGraphs/*/All/graph_*.json,
    roda a avaliação e salva CSVs por análise + agregados globais.
    """
    root = Path(root)
    all_fp_list = []
    all_frames_list = []

    json_files = sorted(root.glob("*/*/graph_*.json"))  # ex: 0/All/graph_0.json
    if not json_files:
        print(f"Nenhum JSON encontrado em {root}/**/graph_*.json")
        return None, None

    for json_path in json_files:
        # pair_id é o diretório logo abaixo de CrossGraphs
        try:
            pair_id = json_path.parents[1].name  # .../CrossGraphs/<pair_id>/All/graph_*.json
        except Exception:
            pair_id = "UNKNOWN"
        print(f"▶ Processando {pair_id}: {json_path.name}")

        try:
            df_fp_nodes, df_frames_nodes_w = process_one_json(json_path)
        except Exception as e:
            print(f"  [ERRO] Falhou em {json_path}: {e}")
            continue

        if df_fp_nodes.empty and df_frames_nodes_w.empty:
            print("  [AVISO] Avaliação vazia, pulando.")
            continue

        # anotar o pair_id nas tabelas
        if not df_fp_nodes.empty:
            df_fp_nodes.insert(0, "pair_id", pair_id)
            all_fp_list.append(df_fp_nodes)

        if not df_frames_nodes_w.empty:
            df_frames_nodes_w.insert(0, "pair_id", pair_id)
            all_frames_list.append(df_frames_nodes_w)

        # salvar CSVs por análise
        save_pair_outputs(json_path.parents[1], df_fp_nodes, df_frames_nodes_w)
        print(f"  ✓ Salvo em {json_path.parents[1] / 'All'}")

    # agregados globais
    df_all_fp = pd.concat(all_fp_list, ignore_index=True) if all_fp_list else pd.DataFrame()
    df_all_frames = pd.concat(all_frames_list, ignore_index=True) if all_frames_list else pd.DataFrame()

    if not df_all_fp.empty:
        df_all_fp.to_csv(root / "ALL_nodes_per_protein.csv", index=False)
    if not df_all_frames.empty:
        # reordenar para pair_id, component_id, frame_id primeiro
        cols = ["pair_id","component_id","frame_id"] + [c for c in df_all_frames.columns if c not in ("pair_id","component_id","frame_id")]
        df_all_frames = df_all_frames[cols]
        df_all_frames.to_csv(root / "ALL_nodes_summary_weighted.csv", index=False)

    print("\n✅ Concluído.")
    print(f"  - Per-protein global: {root/'ALL_nodes_per_protein.csv'}  ({len(df_all_fp)} linhas)")
    print(f"  - Summary   global:   {root/'ALL_nodes_summary_weighted.csv'} ({len(df_all_frames)} linhas)")
    return df_all_fp, df_all_frames

In [13]:
df_all_fp, df_all_frames = batch_evaluate_all("Analysis/CrossGraphs")


▶ Processando 0: graph_0.json
  ✓ Salvo em Analysis/CrossGraphs/0/All
▶ Processando 1: graph_1.json
  ✓ Salvo em Analysis/CrossGraphs/1/All
▶ Processando 10: graph_10.json
  ✓ Salvo em Analysis/CrossGraphs/10/All
▶ Processando 11: graph_11.json
  ✓ Salvo em Analysis/CrossGraphs/11/All
▶ Processando 12: graph_12.json
  ✓ Salvo em Analysis/CrossGraphs/12/All
▶ Processando 13: graph_13.json
  ✓ Salvo em Analysis/CrossGraphs/13/All
▶ Processando 14: graph_14.json
  ✓ Salvo em Analysis/CrossGraphs/14/All
▶ Processando 15: graph_15.json
  ✓ Salvo em Analysis/CrossGraphs/15/All
▶ Processando 16: graph_16.json
  ✓ Salvo em Analysis/CrossGraphs/16/All
▶ Processando 17: graph_17.json
  ✓ Salvo em Analysis/CrossGraphs/17/All
▶ Processando 18: graph_18.json
  ✓ Salvo em Analysis/CrossGraphs/18/All
▶ Processando 2: graph_2.json
  ✓ Salvo em Analysis/CrossGraphs/2/All
▶ Processando 20: graph_20.json
  ✓ Salvo em Analysis/CrossGraphs/20/All
▶ Processando 21: graph_21.json
  ✓ Salvo em Analysis/CrossG

In [15]:
def run_pairwise_per_group(crossDf, args, association_config, root="Analysis/CrossGraphs"):
    """
    Para cada TCR_pair_id (grupo), roda AssociatedGraph par-a-par (2 a 2),
    salva o JSON de cada par e constrói matrizes de similaridade do grupo.
    Retorna um dict: pair_id -> dict com dataframes das matrizes.
    """
    out = {}
    TCR_pair_id = crossDf.groupby('TCR_pair_id')

    for pair_id, group in TCR_pair_id:
        print(f"\n=== Grupo {pair_id} ===")
        refs = list(group["PDB_ID"])
        refs_norm = [r.lower() for r in refs]
        # nomes de arquivo padronizados
        files_map = {r: f"noTCR_{r.lower()}.trunc.fit_renum.pdb" for r in refs}

        # preparar matrizes
        idx = refs
        M_max = pd.DataFrame(0.0, index=idx, columns=idx)
        M_mean = pd.DataFrame(0.0, index=idx, columns=idx)

        # cria destino do grupo
        group_dir = Path(root) / str(pair_id)
        (group_dir / "Pairs").mkdir(parents=True, exist_ok=True)

        for r1, r2 in combinations(refs, 2):
            files_name = ",".join([files_map[r1], files_map[r2]])
            args.files_name = files_name
            args.run_name = f"{pair_id}_{r1}_{r2}"
            args.output_path = str(group_dir / "Pairs" / f"{r1}_{r2}")
            Path(args.output_path).mkdir(parents=True, exist_ok=True)

            graphs = create_graphs(args)
            try:
                G = AssociatedGraph(
                    graphs=graphs,
                    output_path=args.output_path,
                    run_name=args.run_name,
                    association_config=association_config
                )
            except Exception as e:
                print(f"[SKIP] Falha no par ({r1},{r2}): {e}")
                continue

            # opcional: salvar figuras do grafo associado
            G.draw_graph_interactive(show=False, save=True)

            # montar JSON no mesmo formato que você já usava
            graphs_raw = G.graph_data
            associated_graph_data = {"original_graphs": {}}

            for graph_raw in graphs_raw:
                pdb_file = graph_raw["pdb_file"]
                _id = graph_raw["id"]
                # extrai nome 4 letras
                m = re.search(r'noTCR_([A-Za-z0-9]{4})\.trunc', pdb_file, re.IGNORECASE)
                name = m[1] if m else f"id{_id}"

                nodes = list(graph_raw["graph"].nodes)
                edges = list(graph_raw["graph"].edges)
                neighbors = {str(node): [str(nei) for nei in graph_raw["graph"].neighbors(node)] for node in nodes}

                associated_graph_data["original_graphs"][_id] = {
                    "name": name,
                    "nodes": nodes,
                    "edges": edges,
                    "neighbors": neighbors
                }

            for j, comps in enumerate(G.associated_graphs):
                associated_graph_data[j] = {"comp": j, "frames": {}}
                for i in range(len(comps[0])):
                    nodes = list(comps[0][i].nodes)
                    edges = list(comps[0][i].edges)
                    neighbors = {str(node): [str(nei) for nei in comps[0][i].neighbors(node)] for node in nodes}
                    associated_graph_data[j]["frames"][i] = {"nodes": nodes, "edges": edges, "neighbors": neighbors}

            json_path = Path(args.output_path) / f"graph_{pair_id}_{r1}_{r2}.json"
            with open(json_path, "w") as f:
                json.dump(associated_graph_data, f, indent=4)
            print(f"  • JSON salvo: {json_path}")

            # --- AVALIAÇÃO (nós) e RESUMO por frame para gerar a nota do PAR ---
            df_fp_nodes, df_frames_nodes_w = evaluate_all_frames_nodes_weighted(str(json_path))

            # nota do par: escolha como representar (duas opções úteis)
            # 1) melhor frame do par (max do node_cov_wmean)
            max_score = df_frames_nodes_w["node_cov_wmean"].max() if not df_frames_nodes_w.empty else 0.0
            # 2) média dos frames do par
            mean_score = df_frames_nodes_w["node_cov_wmean"].mean() if not df_frames_nodes_w.empty else 0.0

            M_max.loc[r1, r2] = M_max.loc[r2, r1] = max_score
            M_mean.loc[r1, r2] = M_mean.loc[r2, r1] = mean_score
            # diagonal = 1.0 por definição
            M_max.loc[r1, r1] = M_max.loc[r2, r2] = 1.0
            M_mean.loc[r1, r1] = M_mean.loc[r2, r2] = 1.0

        # salva matrizes do grupo
        M_max.to_csv(group_dir / f"matrix_nodes_wmean_MAX.csv")
        M_mean.to_csv(group_dir / f"matrix_nodes_wmean_MEAN.csv")
        print(f"  ✓ Matrizes salvas em {group_dir}")

        out[pair_id] = {"max": M_max, "mean": M_mean}

    return out


In [16]:
result = run_pairwise_per_group(crossDf, args, association_config, root="Analysis/CrossGraphs")


=== Grupo 0 ===


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
  • JSON salvo: Analysis/CrossGraphs/0/Pairs/4MS8_4MXQ/graph_0_4MS8_4MXQ.json


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
  • JSON salvo: Analysis/CrossGraphs/0/Pairs/4MS8_4N0C/graph_0_4MS8_4N0C.json


...

Subgraph exposed_residues created with success!
  • JSON salvo: Analysis/CrossGraphs/35/Pairs/8EO8_8ENH/graph_35_8EO8_8ENH.json


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
  • JSON salvo: Analysis/CrossGraphs/35/Pairs/8EO8_8EN8/graph_35_8EO8_8EN8.json


Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
  • JSON salvo: Analysis/CrossGraphs/35/Pairs/8ENH_8EN8/graph_35_8ENH_8EN8.json
  ✓ Matrizes salvas em Analysis/CrossGraphs/35
